In [ ]:
import yaml
import astropy.units as u
import numpy as np
import matplotlib.pyplot as plt
from SimBMVtool import SimBMVtool as sim

path_to_gammapy_data = '<path_to_local_gammapy_data_copy>'
path_config = './config_simu.yaml'

To try this notebook with example data please check the README

In [ ]:
# For the simulation, SimBMVtool needs FULL-ENCLOSURE dl3 files with at least the exposure, edisp and aeff IRFs.

# In the config file, set the path_data and the run numbers of the used observations.
# If you don't already know which one you can use, check the obs_ids in the data store


path_to_data = f'{path_to_gammapy_data}/dev/hess-dl3-dr1/data'
data_store = sim.get_data_store(path_to_data,'*_dl3_*.fits.gz')
display(data_store.obs_table[data_store.obs_table['OBJECT']=='Crab Nebula'])

In [ ]:
accmodel = sim.BaseSimBMVCreator()
accmodel.load_true_background_irfs(path_config) 

# If you want to get the gammapy BackgroundIRF without knowing the name of the variables
true = accmodel.get_background_irf(type='true',downsampled=False) 
true.plot_at_energy(["1 TeV"],figsize=(3,3))
plt.show()

# You can also plot directly from the class object.
if accmodel.true_collection: accmodel.bkg_true_irf[0].peek()
else: accmodel.bkg_true_irf.peek()

In [ ]:
do_simulation = True
if do_simulation: accmodel.do_simulation() # Simulating gives a lot of warning, this is expected
else: accmodel.load_observation_collection()

In [ ]:
accmodel.obs_collection[1].bkg.peek()
accmodel.obs_collection[1].events.peek()